# Seminario de titulo

## Crear soluciones (no ejecutar si no es necesario)

### Crear Instancias

In [3]:
from clp.get_instance import get_uni

get_uni("instances",10,100)

### Crear soluciones solver

In [2]:
import sys
sys.path.append("c:/Users/franc/Escritorio/Trabajo-final-de-titulo/src")

from solver.get_solver_solutions import generarSolucionesSolver

generarSolucionesSolver(100)

Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 0 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 1 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 2 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 3 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 4 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 5 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 6 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 7 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/s